In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open("intents.json", mode='r', encoding="utf-8") as file:
    data = json.load(file)

training_sentences = [] # contains all training data
training_labels = []    # holds target labels corresponding to each training data

labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [3]:
# Encode labels
label_encoder = LabelEncoder()
label_encoder.fit(training_labels)
encoded_training_labels = label_encoder.transform(training_labels)
print('encoded labels: ', list(zip(training_labels, encoded_training_labels)))

encoded labels:  [('greeting', 4), ('greeting', 4), ('greeting', 4), ('greeting', 4), ('greeting', 4), ('goodbye', 3), ('goodbye', 3), ('goodbye', 3), ('thanks', 7), ('thanks', 7), ('thanks', 7), ('thanks', 7), ('about', 0), ('about', 0), ('about', 0), ('name', 6), ('name', 6), ('name', 6), ('help', 5), ('help', 5), ('help', 5), ('help', 5), ('help', 5), ('help', 5), ('help', 5), ('createaccount', 2), ('createaccount', 2), ('createaccount', 2), ('createaccount', 2), ('createaccount', 2), ('complaint', 1), ('complaint', 1), ('complaint', 1)]


In [4]:
# vectorize data corpus by using Tokenizer to limit vocabulary size
# all non-word characters will be removed

vocab_size = 1000
embedding_dimension = 16
max_pad_seq_len = 20
oov_token = "<OOV>" # to deal with out of vocabulary tokens

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# space-separated sequences of words, split into list of tokens
sequences = tokenizer.texts_to_sequences(training_sentences)
print('tokenized training sentences: ', list(zip(training_sentences, sequences)))

# pad_sequences is used to make all the training text sequences into the same size
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_pad_seq_len)

tokenized training sentences:  [('Hi', [29]), ('Hey', [30]), ('Is anyone there?', [10, 31, 18]), ('Hello', [32]), ('Hay', [33]), ('Bye', [34]), ('See you later', [35, 2, 36]), ('Goodbye', [37]), ('Thanks', [19]), ('Thank you', [38, 2]), ("That's helpful", [39, 40]), ('Thanks for the help', [19, 11, 41, 8]), ('Who are you?', [20, 12, 2]), ('What are you?', [9, 12, 2]), ('Who you are?', [20, 2, 12]), ('what is your name', [9, 10, 21, 22]), ('what should I call you', [9, 42, 4, 43, 2]), ('whats your name?', [44, 21, 22]), ('Could you help me?', [45, 2, 8, 5]), ('give me a hand please', [46, 5, 3, 47, 23]), ('Can you help?', [13, 2, 8]), ('What can you do for me?', [9, 13, 2, 48, 11, 5]), ('I need a support', [4, 14, 3, 24]), ('I need a help', [4, 14, 3, 8]), ('support me please', [24, 5, 23]), ('I need to create a new account', [4, 14, 6, 15, 3, 16, 7]), ('how to open a new account', [25, 6, 26, 3, 16, 7]), ('I want to create an account', [4, 27, 6, 15, 28, 7]), ('can you create an accoun

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dimension, input_length=max_pad_seq_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# display model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [6]:
epochs = 1000
encoded_training_labels_tensor = tf.convert_to_tensor(encoded_training_labels)
print(encoded_training_labels_tensor)

history = model.fit(padded_sequences, encoded_training_labels_tensor, epochs=epochs)


tf.Tensor([4 4 4 4 4 3 3 3 7 7 7 7 0 0 0 6 6 6 5 5 5 5 5 5 5 2 2 2 2 2 1 1 1], shape=(33,), dtype=int32)
Epoch 1/1000
2/2 [==============================] - 0s 3ms/step - loss: 2.0814 - accuracy: 0.0909
Epoch 2/1000
2/2 [==============================] - 0s 2ms/step - loss: 2.0801 - accuracy: 0.0909
Epoch 3/1000
2/2 [==============================] - 0s 998us/step - loss: 2.0798 - accuracy: 0.0909
Epoch 4/1000
2/2 [==============================] - 0s 2ms/step - loss: 2.0796 - accuracy: 0.0909
Epoch 5/1000
2/2 [==============================] - 0s 2ms/step - loss: 2.0794 - accuracy: 0.0909
Epoch 6/1000
2/2 [==============================] - 0s 1ms/step - loss: 2.0791 - accuracy: 0.1515
Epoch 7/1000
2/2 [==============================] - 0s 1ms/step - loss: 2.0789 - accuracy: 0.0909
Epoch 8/1000
2/2 [==============================] - 0s 2ms/step - loss: 2.0788 - accuracy: 0.0909
Epoch 9/1000
2/2 [==============================] - 0s 998us/step - loss: 2.0788 - accuracy: 0.0909
Epoch 10/

In [7]:
# save the trained model
model.save("chat_model")

import pickle

with open('tokenizer.pickle', 'wb') as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as file:
    pickle.dump(label_encoder, file, protocol=pickle.HIGHEST_PROTOCOL)


INFO:tensorflow:Assets written to: chat model\assets
